In [38]:
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [39]:
def draw_natural_line(draw, x1, y1, x2, y2):
    """
    Естественный алгоритм рисования прямой 
    """
    dx = x2 - x1
    dy = y2 - y1
    steps = max(abs(dx), abs(dy))
    x_inc = dx / float(steps)
    y_inc = dy / float(steps)
    x = x1
    y = y1
    for _ in range(steps + 1):
        draw.point((round(x), round(y)), fill="black")
        x += x_inc
        y += y_inc


In [40]:
def draw_bresenham_circle(draw, xc, yc, radius):
    """
    Алгоритм Брезенхама для рисования окружности 
    """
    x = radius
    y = 0
    p = 1 - radius  # Начальное значение
    while x > y:
        # Рисуем 8 симметричных точек
        draw.point((xc + x, yc + y), fill="black")
        draw.point((xc - x, yc + y), fill="black")
        draw.point((xc + x, yc - y), fill="black")
        draw.point((xc - x, yc - y), fill="black")
        draw.point((xc + y, yc + x), fill="black")
        draw.point((xc - y, yc + x), fill="black")
        draw.point((xc + y, yc - x), fill="black")
        draw.point((xc - y, yc - x), fill="black")
        y += 1
        if p <= 0:
            p = p + 2 * y + 1
        else:
            x -= 1
            p = p + 2 * y - 2 * x + 1


In [41]:
def draw_bezier_curve(draw, p0, p1, p2, p3, num_points=100):
    """
    Геометрический способ построения кривых Безье
    """
    for t in range(num_points + 1):
        t /= num_points
        x = int((1 - t) ** 3 * p0[0] + 3 * (1 - t) ** 2 * t * p1[0] +
                 3 * (1 - t) * t ** 2 * p2[0] + t ** 3 * p3[0])
        y = int((1 - t) ** 3 * p0[1] + 3 * (1 - t) ** 2 * t * p1[1] +
                 3 * (1 - t) * t ** 2 * p2[1] + t ** 3 * p3[1])
        draw.point((x, y), fill="black")


In [42]:
def flood_fill(image, x, y, target_color, replacement_color):
    """
    Короед
    """
    pixels = image.load()
    if pixels[x, y] != target_color:
        return
    
    stack = [(x, y)]
    while stack:
        x, y = stack.pop()
        if pixels[x, y] == target_color:
            pixels[x, y] = replacement_color
            stack.extend([(x + dx, y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]])


In [13]:
def patterned_fill(image, x, y, target_color, pattern):
    """
    Модифицированный рекурсивный алгоритм с "затравкой"
    """
    pixels = image.load()
    width, height = pattern.size
    px_pattern = pattern.load()
    
    def recursive_fill(x, y):
        if pixels[x, y] != target_color:
            return
        pixels[x, y] = px_pattern[x % width, y % height]
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < image.width and 0 <= ny < image.height:
                recursive_fill(nx, ny)
    
    recursive_fill(x, y)


In [45]:
img_size = (200, 200)
house_img = Image.new("RGB", img_size, "white")
draw = ImageDraw.Draw(house_img)
house_img.show()
# Рисуем стены
# draw_natural_line(draw, 50, 50, 50, 150)  # Левая стена
# draw_natural_line(draw, 50, 150, 150, 150)  # Нижняя стена
# draw_natural_line(draw, 150, 150, 150, 50)  # Правая стена
# draw_natural_line(draw, 150, 50, 50, 50)  # Верхняя стена

# # Рисуем окна (круги)
# draw_bresenham_circle(draw, 75, 125, 10)  # Левое окно
# draw_bresenham_circle(draw, 125, 125, 10)  # Правое окно

# # Рисуем крышу
# draw_natural_line(draw, 50, 50, 100, 20)  # Левый скат крыши
# draw_natural_line(draw, 100, 20, 150, 50)  # Правый скат крыши

# # Закраска стен с помощью алгоритма "короеда"
# flood_fill(house_img, 55, 55, house_img.getpixel((55, 55)), (255, 200, 200))

# # Создаем узор для закраски
# pattern = Image.new("RGB", (10, 10), "white")
# pattern_draw = ImageDraw.Draw(pattern)
# pattern_draw.line((0, 0, 10, 10), fill="gray", width=1)
# pattern_draw.line((0, 10, 10, 0), fill="gray", width=1)

# # Закраска стен с узором
# patterned_fill(house_img, 55, 55, (255, 200, 200), pattern)

# # Отображаем результат
# house_img.show()